In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import argparse

import torch
#from tokenization_hanbert import HanBertTokenizer
import pandas as pd
import numpy as np
import collections
import h5py
#from dataloader import KoreanDataset, KoreanBatchGenerator, get_dataloader
from tqdm.notebook import tqdm
import multiprocessing

from make_h5py import sharding_h5py

In [11]:
#path = 'bart_corpus/'
output_path ='sharding_corpus/'

#num_divided=374439
#ip=2536266

num_divided=281600
ip_lengths=int(4700000/2)


In [12]:
ip_lengths

2350000

In [13]:
def write_single_shard(shard_name, file):
    with open(shard_name, mode='w') as f:
        f.writelines(file)  # Line break between articles

In [14]:
from google.cloud import storage
client = storage.Client()
bucket = storage.Bucket(client, "hanbart_bucket")
all_blobs = list(client.list_blobs(bucket, prefix='bart_corpus_512'))

In [18]:
all_blobs[12]

<Blob: hanbart_bucket, bart_corpus_512/bartip-2.selected.txt, 1635473634037316>

In [19]:
lines = []
shard_file_index=0

for blob_index, blob in enumerate(all_blobs):
    print(blob.name, blob_index)
    with blob.open("rt") as f:
        if blob_index==11 or blob_index==12:
            print(f'mount of {blob.name} is {ip_lengths} (cur lines is {len(lines)})')
            lines.extend(f.readlines()[:ip_lengths])
        else:
            lines.extend(f.readlines())
    print(len(lines))

    while(len(lines) > num_divided):
        write_single_shard(output_path+f'bartcorpus_{str(shard_file_index).zfill(3)}.txt', lines[:num_divided])
        del lines[:num_divided]
        shard_file_index+=1
write_single_shard(output_path+f'bartcorpus_{str(shard_file_index).zfill(3)}.txt', lines)        

bart_corpus_512/bart100dic_doosan.step2.txt 0
231254
bart_corpus_512/bart100dic_pascal.step2.txt 1
380625
bart_corpus_512/bartNIKL_NEWSPAPER.txt 2
25245151
bart_corpus_512/bartNIKL_WRITTEN.txt 3
12045193
bart_corpus_512/bartblog1-out.txt 4
1000272
bart_corpus_512/bartblog2-out.txt 5
167416
bart_corpus_512/bartbooks-out.txt 6
1461852
bart_corpus_512/bartcafe-out.txt 7
374760
bart_corpus_512/bartcampus-out.txt 8
320620
bart_corpus_512/bartclinic-out.txt 9
40135
bart_corpus_512/bartdart-10.12.out.txt 10
3292704
bart_corpus_512/bartip-1.selected.txt 11
mount of bart_corpus_512/bartip-1.selected.txt is 2350000 (cur lines is 195104)
2545104
bart_corpus_512/bartip-2.selected.txt 12
mount of bart_corpus_512/bartip-2.selected.txt is 2350000 (cur lines is 10704)
2360704
bart_corpus_512/bartkowiki-2019-out.txt 13
1589120
bart_corpus_512/bartlaw-out.txt 14
207375
bart_corpus_512/bartlife-out.txt 15
292879
bart_corpus_512/bartnamu-2019-out.txt 16
2682665
bart_corpus_512/bartnews-1997-out.txt 17
564

# Make h5py file

In [26]:
import subprocess


In [27]:
shard_file_index=0

In [28]:
sharding_path = '/home/jisu/sharding_corpus/'
h5py_path = "/home/jisu/h5py_dataset/"


In [34]:
files = os.listdir(sharding_path)

In [46]:
n_processes = 8
def create_record_worker(shard_file_index):
    sharding_filename = f"bartcorpus_{str(shard_file_index).zfill(3)}.txt"
    h5py_filename = f"h5py_data_{str(shard_file_index).zfill(3)}"

    bart_preprocessing_command = 'python make_h5py.py'
    bart_preprocessing_command += ' --input_file=' + sharding_path+sharding_filename
    bart_preprocessing_command += ' --output_file=' + h5py_path+h5py_filename+'.h5py'

    bert_preprocessing_process = subprocess.Popen(bart_preprocessing_command, shell=True)

    last_process = bert_preprocessing_process

    # This could be better optimized (fine if all take equal time)
    if shard_file_index % n_processes == 0 and shard_file_index > 0:
        bert_preprocessing_process.wait()
    return last_process

for i in range(1, len(files)):
    last_process =create_record_worker(i)

last_process.wait()


1